In [5]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [6]:
market = Market()
sec = SEC()
fin = ADatabase("dividends")
umod = UniversalModeler()

In [7]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [16]:
factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity',
"commonstockdividendspersharecashpaid"]

In [9]:
included_columns=[
    "year",
    "quarter",
    "ticker",
    "adjclose",
    "y"
]

In [10]:
included_columns.extend(factors)

In [17]:
market.connect()
sec.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        filing = sec.retrieve_filing_data(cik)
        filing = p.column_date_processing(filing)
        filing = filing.groupby(["year","quarter"]).mean().reset_index()
        ticker_data = prices.copy()
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","quarter"]).mean().reset_index()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data = ticker_data.merge(filing,on=["year","quarter"],how="left").reset_index()
        ticker_data["y"] = ticker_data["adjclose"].shift(-4) + ticker_data["commonstockdividendspersharecashpaid"]
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

  1%|█                                                                                                                                                                          | 3/490 [00:00<02:24,  3.36it/s]

'commonstockdividendspersharecashpaid'


  1%|█▋                                                                                                                                                                         | 5/490 [00:01<02:02,  3.97it/s]

'commonstockdividendspersharecashpaid'


  1%|██                                                                                                                                                                         | 6/490 [00:01<02:00,  4.01it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 488/490 [02:58<00:00,  2.80it/s]

'commonstockdividendspersharecashpaid'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 489/490 [02:59<00:00,  1.82it/s]

"['cashandcashequivalentsatcarryingvalue'] not in index"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [03:00<00:00,  2.72it/s]


In [18]:
data = pd.concat(training_sets)

In [19]:
data = data.dropna()

In [20]:
data

,year,quarter,ticker,adjclose,y,assets,liabilitiesandstockholdersequity,incometaxexpensebenefit,retainedearningsaccumulateddeficit,accumulatedothercomprehensiveincomelossnetoftax,earningspersharebasic,earningspersharediluted,propertyplantandequipmentnet,cashandcashequivalentsatcarryingvalue,entitycommonstocksharesoutstanding,weightedaveragenumberofdilutedsharesoutstanding,weightedaveragenumberofsharesoutstandingbasic,stockholdersequity,commonstockdividendspersharecashpaid,quarter
48,2012,1,MMM,62.705929,77.472781,2.967400e+10,3.088600e+10,1.551333e+09,2.717150e+10,-4.284000e+09,2.555455,2.519091,7.472500e+09,2.621250e+09,694543763.0,7.170667e+08,7.075667e+08,1.554150e+10,2.113333,1
49,2012,2,MMM,62.787037,81.307970,3.181550e+10,3.181550e+10,4.520000e+08,2.860300e+10,-4.905000e+09,1.565000,1.540000,7.710500e+09,2.504800e+09,693872048.0,7.162500e+08,7.041500e+08,1.580100e+10,0.570000,2
50,2012,3,MMM,66.710493,87.814722,3.247750e+10,3.247750e+10,6.990000e+08,2.890650e+10,-5.018500e+09,2.440000,2.400000,7.686000e+09,3.070000e+09,691319463.0,7.149750e+08,7.039250e+08,1.591300e+10,0.855000,3
51,2012,4,MMM,67.131673,96.917189,3.304750e+10,3.304750e+10,9.145000e+08,2.924900e+10,-4.786500e+09,3.217500,3.172500,7.802500e+09,3.000250e+09,691931278.0,7.113250e+08,7.015750e+08,1.653200e+10,1.140000,4
52,2013,1,MMM,75.359447,102.721556,3.188267e+10,3.274600e+10,1.702000e+09,2.951350e+10,-4.887500e+09,2.783636,2.743636,8.022000e+09,2.879750e+09,689990255.0,7.159333e+08,7.053667e+08,1.649750e+10,2.220000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2014,3,ZTS,32.217975,44.032564,6.540000e+09,6.540000e+09,9.100000e+07,4.035000e+08,-2.090000e+08,0.412500,0.412500,1.302000e+09,4.685000e+08,501193395.0,5.008145e+08,5.003945e+08,1.100000e+09,0.066750,3
8,2015,1,ZTS,42.616156,40.601214,6.582500e+09,6.582500e+09,1.012727e+08,4.925000e+08,-2.005000e+08,0.473636,0.473636,1.306500e+09,4.720000e+08,500787817.0,5.007807e+08,5.003523e+08,1.125500e+09,0.188667,1
12,2016,1,ZTS,40.412547,52.181372,7.250500e+09,7.250500e+09,9.681818e+07,7.925000e+08,-3.397500e+08,0.426364,0.426364,1.312500e+09,7.407500e+08,497155532.0,5.014537e+08,5.002547e+08,1.189500e+09,0.303333,1
16,2017,1,ZTS,51.878038,76.496562,7.781000e+09,7.781000e+09,1.330000e+08,1.176500e+09,-6.100000e+08,0.530000,0.529091,1.344000e+09,8.432500e+08,491964064.0,5.007563e+08,4.988257e+08,1.277500e+09,0.344333,1


In [21]:
fin.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2016,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors,tf=False)
        stuff["training_years"] = years
        fin.store(set_name,stuff)
fin.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:57<00:00,  8.28s/it]


In [22]:
stuff

,year,quarter,ticker,adjclose,y,assets,liabilitiesandstockholdersequity,incometaxexpensebenefit,retainedearningsaccumulateddeficit,accumulatedothercomprehensiveincomelossnetoftax,...,weightedaveragenumberofsharesoutstandingbasic,stockholdersequity,commonstockdividendspersharecashpaid,quarter,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction,training_years
0,2022,1,AES,21.809554,25.338703,3.373800e+10,3.378300e+10,1.022500e+08,-6.140000e+08,-2.282000e+09,...,6.650000e+08,2.716000e+09,0.150500,1,24.525713,0.874719,16.042456,0.850922,20.284085,4
1,2022,1,AFL,60.868745,69.427896,1.613140e+11,1.613140e+11,5.063333e+08,3.968250e+10,6.273250e+09,...,7.099110e+08,2.980825e+10,1.173333,1,259.045563,0.874719,109.269951,0.850922,184.157757,4
2,2022,1,A,136.502816,146.156531,1.051600e+10,1.051600e+10,3.000000e+07,2.535000e+08,-2.900000e+08,...,3.035000e+08,5.271500e+09,0.202000,1,90.601097,0.874719,92.416470,0.850922,91.508784,4
3,2022,1,ALLE,116.370657,113.099574,3.029200e+09,3.060200e+09,5.490000e+07,9.691000e+08,-1.984000e+08,...,9.193333e+07,7.942500e+08,1.266667,1,137.584885,0.874719,158.354928,0.850922,147.969906,4
4,2022,1,LNT,57.115556,54.147285,1.246271e+10,1.175633e+10,-1.577778e+07,1.693000e+09,-5.000000e+05,...,2.457000e+08,4.085333e+09,1.516667,1,64.739983,0.874719,87.152228,0.850922,75.946105,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2022,1,WRK,44.355899,33.097756,2.916840e+10,2.916840e+10,5.445000e+07,1.659550e+09,-1.111750e+09,...,2.636500e+08,1.144350e+10,0.225000,1,42.011883,0.874719,47.937329,0.850922,44.974606,4
66,2022,1,WY,36.948036,32.005284,1.698150e+10,1.698150e+10,2.523333e+08,1.271000e+09,-6.505000e+08,...,7.474413e+08,9.225000e+09,1.016667,1,75.686234,0.874719,95.670372,0.850922,85.678303,4
67,2022,1,WHR,192.755651,146.233493,1.989800e+10,2.036050e+10,4.160000e+08,9.447500e+09,-2.584000e+09,...,6.283333e+07,4.365500e+09,5.016667,1,192.423691,0.874719,175.296696,0.850922,183.860194,4
68,2022,1,GWW,485.266612,643.039215,6.443500e+09,6.443500e+09,2.923333e+08,9.139500e+09,-7.850000e+07,...,5.336667e+07,1.851000e+09,6.003333,1,468.661377,0.874719,464.945793,0.850922,466.803585,4
